In [41]:
notebook_id = "121"

<a id="ID_top"></a>
## Country data organisation

This workflow generates (part1), maintains (part2) and exports (part3)

#### Notebook sections:
    
|| [0| Default imports](#ID_top) || [1|Part1 Reference table generation](#ID_part1) || [2|Part2 Table maintanence](#ID_part2) || [3|Part3 Table export](#ID_part3) || 

#### Import all packages that could be required

In [3]:
# %load s_package_import.py
# package library, use to ensure consistency across notebooks, refresh periodically
# general packages
import os # use with os.listdir(_path_)
import requests
import csv
import time
from datetime import datetime
from shutil import copyfile

#temp check
#from shutil import make_archive
import zipfile #notebook

# data analysis packages
import pandas as pd
pd.options.display.max_columns = None # don't truncate columns
#pd.options.display.max_rows = None

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes
import pycountry

# custom scripts
import s_file_export
import s_filepaths
import s_un_comtrade_extract as s_un
import s_adj_matrix_plot

#=== network analysis
import networkx as nx
#=== gavity modelling
import gme as gme

#=== distance datasets
import wbdata


#### Import module and declare path variables
`import s_filepaths.py`

In [4]:
# import ref file
import s_filepaths

# declare local variables to work with
path_raw = s_filepaths.path_raw
path_raw_dl = s_filepaths.path_raw_dl
path_store = s_filepaths.path_store
path_live = s_filepaths.path_live

In [ ]:
#os.listdir(path_livepath_live)

****

<a id="ID_part1"></a>
### Part 1 | Import all countries and set scope
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

**Import master country reference file**

In [5]:
# View all files in live directory
print(os.listdir(f"{path_live}"))

['.DS_Store', '112_input_gme_data.csv.gzip', '120_input_scope_data.csv.gzip', '2_raw_explainer_doc.md', '113_input_bri_members.csv.gzip']


In [6]:
country_file = "120_input_scope_data.csv.gzip"
df_scope = pd.read_csv(f"{path_live}{country_file}",compression="gzip")
df_scope.head()

,year,country_d,iso3_d,dynamic_code_d,landlocked_d,island_d,region_d,gdp_pwt_const_d,pop_d,lat_d,lng_d,country_o,iso3_o,dynamic_code_o,landlocked_o,island_o,region_o,pop_o,lat_o,lng_o,contiguity,distance,bri_year_d,bri_flag_d
0,2016,United Arab Emirates,ARE,ARE,0,0,middle_east,NaN,NaN,25.094378,55.454674,Portugal,PRT,PRT,0,0,europe,NaN,39.310741,-10.883486,0,6154.421400,2017.0,1.0
1,2016,Armenia,ARM,ARM,1,0,europe,NaN,NaN,40.379307,44.671028,Armenia,ARM,ARM,1,0,europe,NaN,40.379307,44.671028,0,27.876888,2015.0,1.0
2,2016,Azerbaijan,AZE,AZE,0,0,europe,NaN,NaN,40.362438,47.255833,Kuwait,KWT,KWT,0,0,middle_east,NaN,29.279369,47.930035,0,1236.608600,2015.0,1.0
3,2016,Belgium,BEL,BEL,0,0,europe,NaN,NaN,50.691814,4.581812,Cambodia,KHM,KHM,0,0,south_east_asia,NaN,12.320902,104.874400,0,9693.062500,NaN,NaN
4,2016,China,CHN,CHN,0,0,east_asia,NaN,NaN,35.389668,114.002470,Belarus,BLR,BLR,1,0,europe,NaN,53.604687,27.802185,0,6904.336400,2013.0,1.0


In [11]:
# get country list
list_scope_countries = list(df_scope.iso3_d.unique())
len(list_scope_countries)

88

Check countries are present in the World Bank database.

In [15]:
def f_wb_iso3_check(p_iso3_list):
    wb_country_list = []
    # Check presence in WB database
    for iso3 in p_iso3_list:
        try:
            wbdata.api.get_country(iso3)
            wb_country_list.append(iso3)
        except:
            print(f"{iso3} gave error")

    print(f"\n{len(wb_country_list)} are included in the refined list.")
    return wb_country_list

In [16]:
list_wb_scope = f_wb_iso3_check(list_scope_countries)

GAZ gave error
VAT gave error

86 are included in the refined list.


<a id="ID_part2"></a>
### Part 2 | Download data
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

**This section extracts data for:**<br>
**A.** Economic distance (WDI) - source 2<br>
**B.** Institutional distance (WGI) - source 3
<br><br>
Both are sourced from the World Bank. 

In [20]:
# Master variables
target_date = datetime(2016,1,1)

**A | Economic Distance (DE) | Download**

In [22]:
# Economic distance indicators (DE) based on WDI
de_indicator = ["NY.GDP.PCAP.PP.CD"]
target_date = target_date

In [31]:
# Iterate over country and indicator to correctly flag what is available
outputs = []
list_excluded = []
df_wb_de = pd.DataFrame()

# for every country in list of available countries for our desired region
for country in list_wb_scope:
    
    # iterate through the indicators one by one
    try:
        for indicator in de_indicator:
            df_wb_de.loc[country,indicator] = wbdata.api.get_series(indicator = indicator, source = "2",
                                                     country = country,data_date=target_date)[0] 
    except:
        print(f"WARNING | Country: {country} data not available")
        list_excluded.append(country)

**B | Institutional Distance (DI) | Download**

In [34]:
# Insitutional distance indicators (DI) based on WGI
indicator_list = ["CC.EST","GE.EST","PV.EST","RL.EST","RQ.EST","VA.EST"]
target_date = target_date

In [35]:
# Iterate over country and indicator to correctly flag what is available
outputs = []
list_excluded_di = []
df_wb_di = pd.DataFrame()

# for every country in list of available countries for our desired region
for country in list_wb_scope:
    
    # iterate through the indicators one by one
    try:
        for indicator in indicator_list:
            df_wb_di.loc[country,indicator] = wbdata.api.get_series(indicator = indicator, source = "3",
                                                     country = country,data_date=target_date)[0] 
    except:
        print(f"WARNING | Country: {country} data not available")
        list_excluded_di.append(country)

WARNING | Country: GIB data not available
WARNING | Country: FRO data not available
WARNING | Country: SMR data not available


**A+B | Merge dataframes**

In [39]:
# Merge data
df_wb_all = df_wb_di.merge(df_wb_de,left_index = True, right_index= True).copy()
df_wb_all.head()

# only keep entries with data
df_wb_all.dropna(inplace= True)
missing_country = set(list_wb_scope) - set(list(df_wb_all.index))
print(f"Original list = {len(list_wb_scope)} | Economic and Institutional data for = {len(df_wb_all)}")
print(f"Countries with no data are: {missing_country}")

# rename columns
df_wb_all.columns = ["Control of corruption","Government effectiveness","Political Stability and Absence of Violence",
                     "Rule of Law","Regulatory Quality","Voice and Accountability","GDP per capita (PPP)"]

df_wb_all.head()

Original list = 86 | Economic and Institutional data for = 77
Countries with no data are: {'YEM', 'GIB', 'FRO', 'SMR', 'LIE', 'MCO', 'AND', 'SYR', 'PRK'}


,Control of corruption,Government effectiveness,Political Stability and Absence of Violence,Rule of Law,Regulatory Quality,Voice and Accountability,GDP per capita (PPP)
ARE,1.171709,1.415928,0.564749,0.849041,0.972596,-1.050421,63968.853871
ARM,-0.571955,-0.158175,-0.662264,-0.119130,0.250056,-0.579335,10704.233408
AZE,-0.838504,-0.165613,-0.803879,-0.519861,-0.282023,-1.563101,14371.022741
BEL,1.638705,1.326213,0.441367,1.390965,1.340684,1.376962,48608.957782
CHN,-0.254059,0.354875,-0.499395,-0.334415,-0.264475,-1.561287,13572.620725


In [42]:
# Save to live
filename = "wb_distance_2016"
s_file_export.f_df_export(df_wb_all,filename,p_file_id=f"{notebook_id}_",p_loc2=path_raw,p_loc2_pre="raw_")

Export | ../Data/1_raw_processed_backup/121_store_wb_distance_2016_20200808_2051.csv | COMPLETE
COPY   | ../Data/0_raw/121_raw_wb_distance_2016.csv.gzip | COMPLETE


<a id="ID_part3"></a>
### Part 3 | Calculate martices
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

**This section calculates and manipulates data for:**<br>
**A.** Economic distance (WDI)<br>
**B.** Institutional distance (WGI)
<br><br>
Both are sourced from the World Bank. 

1. A matrix of each country vs each country is created in the order of the columns in table `df_wb_all` and stored in object `distance_matrices`

2. Matrices are separated and the final distance is calculated for each kind of measure.

In [43]:
# Basics variables for matrix to determine size and labels
matrix_size = len(df_wb_all)
matrix_index = list(df_wb_all.index)

In [44]:
# storing object for matrices
distance_matrices = []

# for every column
for column in df_wb_all.columns:
    # create zero matrix
    df_matrix_zero = pd.DataFrame(np.zeros(shape = (matrix_size, matrix_size)),columns = matrix_index,index = matrix_index)
    
    # for every country (index)
    for entry in df_wb_all.index:
        
        for entry_l2 in df_wb_all.index:
            # calculate difference
            temp_difference = df_wb_all.loc[entry,column] - df_wb_all.loc[entry_l2,column]
            
            # assign calculated value to correct position in matrix
            df_matrix_zero.loc[entry,entry_l2] = temp_difference
            
    distance_matrices.append(df_matrix_zero)

**A | Economic distance | calculation**

In [45]:
distance_matrices[6].head()

,ARE,ARM,AZE,BEL,CHN,CZE,ESP,GEO,HUN,IRQ,ITA,KGZ,KOR,KSV,LTU,OMN,SRB,SWE,THA,TJK,UKR,VNM,AFG,ALB,AUT,BGD,BGR,BIH,BLR,BRN,BTN,CHE,DEU,DNK,EST,FIN,FRA,GRC,HKG,HRV,IDN,IND,IRL,IRN,ISR,JOR,KAZ,KHM,KWT,LAO,LBN,LUX,LVA,MAC,MDA,MKD,MMR,MNG,MNE,MYS,NLD,NOR,NPL,PAK,POL,PRT,PSE,QAT,ROU,RUS,SAU,SGP,SVK,TKM,TLS,TUR,UZB
ARE,0.000000,53264.620463,49597.831130,15359.896090,50396.233146,28092.126434,26686.411358,51110.370471,36269.222934,54071.852430,24045.945388,59287.679138,24401.837247,53906.745607,33043.922928,35042.715678,48234.520983,13543.699244,47350.091682,60871.651844,52820.657325,57395.757533,61839.865845,51965.921973,11335.599583,60119.742884,44083.072004,50978.838063,46243.101692,7672.834104,53511.652070,-1751.292658,13404.602563,11998.104071,32737.382725,19038.945094,21048.587053,36145.492027,6747.440108,39250.724189,53474.293405,58128.955282,-7952.863178,49957.308743,26124.876061,54684.983200,40150.029045,60293.815951,19283.503248,57125.292289,48481.744619,-46680.842747,37564.328185,-51729.205366,53358.376029,48985.687397,59476.835550,53226.200779,45883.824085,38422.844643,11685.697022,5034.898095,61385.910198,59558.759118,35685.152434,32364.307560,58473.748341,-19133.631799,39803.101306,39843.455008,18483.194297,-25417.225529,34317.329737,50102.529153,60827.979926,37639.493702,57515.925934
ARM,-53264.620463,0.000000,-3666.789333,-37904.724374,-2868.387317,-25172.494029,-26578.209105,-2154.249992,-16995.397529,807.231967,-29218.675076,6023.058675,-28862.783216,642.125144,-20220.697535,-18221.904786,-5030.099480,-39720.921219,-5914.528781,7607.031381,-443.963138,4131.137070,8575.245382,-1298.698491,-41929.020880,6855.122421,-9181.548459,-2285.782400,-7021.518771,-45591.786360,247.031607,-55015.913121,-39860.017900,-41266.516392,-20527.237738,-34225.675369,-32216.033410,-17119.128436,-46517.180356,-14013.896274,209.672941,4864.334819,-61217.483641,-3307.311721,-27139.744403,1420.362737,-13114.591418,7029.195488,-33981.117215,3860.671826,-4782.875844,-99945.463210,-15700.292278,-104993.825830,93.755565,-4278.933067,6212.215087,-38.419684,-7380.796378,-14841.775820,-41578.923441,-48229.722368,8121.289735,6294.138655,-17579.468029,-20900.312903,5209.127878,-72398.252262,-13461.519157,-13421.165455,-34781.426166,-78681.845992,-18947.290727,-3162.091310,7563.359462,-15625.126761,4251.305470
AZE,-49597.831130,3666.789333,0.000000,-34237.935041,798.402016,-21505.704697,-22911.419772,1512.539341,-13328.608196,4474.021300,-25551.885743,9689.848008,-25195.993883,4308.914477,-16553.908202,-14555.115453,-1363.310147,-36054.131887,-2247.739448,11273.820714,3222.826194,7797.926403,12242.034715,2368.090842,-38262.231547,10521.911754,-5514.759126,1381.006932,-3354.729438,-41924.997027,3913.820940,-51349.123788,-36193.228567,-37599.727059,-16860.448405,-30558.886036,-28549.244078,-13452.339103,-42850.391023,-10347.106941,3876.462274,8531.124152,-57550.694308,359.477612,-23472.955070,5087.152070,-9447.802085,10695.984820,-30314.327882,7527.461159,-1116.086511,-96278.673877,-12033.502945,-101327.036497,3760.544898,-612.143734,9879.004420,3628.369649,-3714.007045,-11174.986487,-37912.134108,-44562.933035,11788.079068,9960.927988,-13912.678696,-17233.523570,8875.917211,-68731.462929,-9794.729824,-9754.376122,-31114.636834,-75015.056659,-15280.501394,504.698023,11230.148795,-11958.337428,7918.094803
BEL,-15359.896090,37904.724374,34237.935041,0.000000,35036.337056,12732.230344,11326.515268,35750.474381,20909.326844,38711.956340,8686.049298,43927.783049,9041.941157,38546.849518,17684.026838,19682.819588,32874.624893,-1816.196846,31990.195592,45511.755754,37460.761235,42035.861444,46479.969755,36606.025883,-4024.296507,44759.846795,28723.175914,35618.941973,30883.205602,-7687.061986,38151.755981,-17111.188747,-1955.293526,-3361.792019,17377.486635,3679.049005,5688.690963,20785.595937,-8612.455982,23890.828099,38114.397315,42769.059193,-23312.759267,34597.412653,10764.979971,39325.087111,24790.132956,44933.919861,3923.607158,41765.396199,33121.848529,-62

In [46]:
# light touch final processing
df_de = distance_matrices[6].copy()
matrix_de = df_de.to_numpy() # create array
matrix_de = np.square(matrix_de) # square all values
df_de = pd.DataFrame(matrix_de,index = matrix_index,columns =matrix_index)
df_de.head()

,ARE,ARM,AZE,BEL,CHN,CZE,ESP,GEO,HUN,IRQ,ITA,KGZ,KOR,KSV,LTU,OMN,SRB,SWE,THA,TJK,UKR,VNM,AFG,ALB,AUT,BGD,BGR,BIH,BLR,BRN,BTN,CHE,DEU,DNK,EST,FIN,FRA,GRC,HKG,HRV,IDN,IND,IRL,IRN,ISR,JOR,KAZ,KHM,KWT,LAO,LBN,LUX,LVA,MAC,MDA,MKD,MMR,MNG,MNE,MYS,NLD,NOR,NPL,PAK,POL,PRT,PSE,QAT,ROU,RUS,SAU,SGP,SVK,TKM,TLS,TUR,UZB
ARE,0.000000e+00,2.837120e+09,2.459945e+09,2.359264e+08,2.539780e+09,7.891676e+08,7.121646e+08,2.612270e+09,1.315457e+09,2.923765e+09,5.782075e+08,3.515029e+09,5.954497e+08,2.905937e+09,1.091901e+09,1.227992e+09,2.326569e+09,1.834318e+08,2.242031e+09,3.705358e+09,2.790022e+09,3.294273e+09,3.824169e+09,2.700457e+09,1.284958e+08,3.614383e+09,1.943317e+09,2.598842e+09,2.138424e+09,5.887238e+07,2.863497e+09,3.067026e+06,1.796834e+08,1.439545e+08,1.071736e+09,3.624814e+08,4.430430e+08,1.306497e+09,4.552795e+07,1.540619e+09,2.859500e+09,3.378975e+09,6.324803e+07,2.495733e+09,6.825091e+08,2.990447e+09,1.612025e+09,3.635344e+09,3.718535e+08,3.263299e+09,2.350480e+09,2.179101e+09,1.411079e+09,2.675911e+09,2.847116e+09,2.399598e+09,3.537494e+09,2.833028e+09,2.105325e+09,1.476315e+09,1.365555e+08,2.535020e+07,3.768230e+09,3.547246e+09,1.273430e+09,1.047448e+09,3.419179e+09,3.660959e+08,1.584287e+09,1.587501e+09,3.416285e+08,6.460354e+08,1.177679e+09,2.510263e+09,3.700043e+09,1.416731e+09,3.308082e+09
ARM,2.837120e+09,0.000000e+00,1.344534e+07,1.436768e+09,8.227646e+06,6.336545e+08,7.064012e+08,4.640793e+06,2.888435e+08,6.516234e+05,8.537310e+08,3.627724e+07,8.330603e+08,4.123247e+05,4.088766e+08,3.320378e+08,2.530190e+07,1.577752e+09,3.498165e+07,5.786693e+07,1.971033e+05,1.706629e+07,7.353483e+07,1.686618e+06,1.758043e+09,4.699270e+07,8.430083e+07,5.224801e+06,4.930173e+07,2.078611e+09,6.102461e+04,3.026751e+09,1.588821e+09,1.702925e+09,4.213675e+08,1.171397e+09,1.037873e+09,2.930646e+08,2.163848e+09,1.963893e+08,4.396274e+04,2.366175e+07,3.747580e+09,1.093831e+07,7.365657e+08,2.017430e+06,1.719925e+08,4.940959e+07,1.154716e+09,1.490479e+07,2.287590e+07,9.989096e+09,2.464992e+08,1.102370e+10,8.790106e+03,1.830927e+07,3.859162e+07,1.476072e+03,5.447616e+07,2.202783e+08,1.728807e+09,2.326106e+09,6.595535e+07,3.961618e+07,3.090377e+08,4.368231e+08,2.713501e+07,5.241507e+09,1.812125e+08,1.801277e+08,1.209748e+09,6.190833e+09,3.589998e+08,9.998821e+06,5.720441e+07,2.441446e+08,1.807360e+07
AZE,2.459945e+09,1.344534e+07,0.000000e+00,1.172236e+09,6.374458e+05,4.624953e+08,5.249332e+08,2.287775e+06,1.776518e+08,2.001687e+07,6.528989e+08,9.389315e+07,6.348381e+08,1.856674e+07,2.740319e+08,2.118514e+08,1.858615e+06,1.299900e+09,5.052333e+06,1.270990e+08,1.038661e+07,6.080766e+07,1.498674e+08,5.607854e+06,1.463998e+09,1.107106e+08,3.041257e+07,1.907180e+06,1.125421e+07,1.757705e+09,1.531799e+07,2.636733e+09,1.309950e+09,1.413739e+09,2.842747e+08,9.338455e+08,8.150593e+08,1.809654e+08,1.836156e+09,1.070626e+08,1.502696e+07,7.278008e+07,3.312082e+09,1.292242e+05,5.509796e+08,2.587912e+07,8.926096e+07,1.144041e+08,9.189585e+08,5.666267e+07,1.245649e+06,9.269583e+09,1.448052e+08,1.026717e+10,1.414170e+07,3.747200e+05,9.759473e+07,1.316507e+07,1.379385e+07,1.248803e+08,1.437330e+09,1.985855e+09,1.389588e+08,9.922009e+07,1.935626e+08,2.969943e+08,7.878191e+07,4.724014e+09,9.593673e+07,9.514785e+07,9.681206e+08,5.627259e+09,2.334937e+08,2.547201e+05,1.261162e+08,1.430018e+08,6.269623e+07
BEL,2.359264e+08,1.436768e+09,1.172236e+09,0.000000e+00,1.227545e+09,1.621097e+08,1.282899e+08,1.278096e+09,4.371999e+08,1.498616e+09,7.544745e+07,1.929650e+09,8.175670e+07,1.485860e+09,3.127248e+08,3.874134e+08,1.080741e+09,3.298571e+06,1.023373e+09,2.071320e+09,1.403309e+09,1.767014e+09,2.160388e+09,1.340001e+09,1.619496e+07,2.003444e+09,8.250208e+08,1.268709e+09,9.537724e+08,5.909092e+07,1.455556e+09,2.927928e+08,3.823173e+06,1.130165e+07,3.019770e+08,1.353540e+07,3.236120e+07,4.320410e+08,7.417440e+07,5.707717e+08,1.452707e+09,1.829192e+09,5.434847e+08,1.196981e+09,1.158848e+08,1.546462e+09,6.145507e+08,2.019057e+09,1.539469e+07,1.744348e+09,1.097057e+09,3.849053e

**B | Institutional distance | calculation**

In [47]:
# list of matrices
list_processed_matrix = []
institutional_variables = distance_matrices[0:6]

# we have five spatial weight matrices which need to be treated together
for index,matrix in enumerate(institutional_variables):
    # square values
    matrix_square = np.square(distance_matrices[0].to_numpy())

    # divide by variance
    matrix_var = distance_matrices[0].to_numpy().var()

    # square divide by var
    matrix_processed = matrix_square / matrix_var
    
    list_processed_matrix.append(matrix_processed)

# sum all spatial matrices 
matrix_di = sum(list_processed_matrix)/len(institutional_variables)

# create df
df_di = pd.DataFrame(matrix_di,index = matrix_index,columns=matrix_index)
df_di.head()

,ARE,ARM,AZE,BEL,CHN,CZE,ESP,GEO,HUN,IRQ,ITA,KGZ,KOR,KSV,LTU,OMN,SRB,SWE,THA,TJK,UKR,VNM,AFG,ALB,AUT,BGD,BGR,BIH,BLR,BRN,BTN,CHE,DEU,DNK,EST,FIN,FRA,GRC,HKG,HRV,IDN,IND,IRL,IRN,ISR,JOR,KAZ,KHM,KWT,LAO,LBN,LUX,LVA,MAC,MDA,MKD,MMR,MNG,MNE,MYS,NLD,NOR,NPL,PAK,POL,PRT,PSE,QAT,ROU,RUS,SAU,SGP,SVK,TKM,TLS,TUR,UZB
ARE,0.000000,1.413471,1.878648,0.101388,0.945060,0.187432,0.200112,0.109969,0.536718,3.043626,0.549619,2.355692,0.237534,1.170081,0.097466,0.321991,1.037939,0.483808,1.132392,2.498508,1.833312,1.226393,3.384270,1.156142,0.066016,1.913548,0.834968,1.232419,0.949055,0.169205,0.000950,0.310050,0.209761,0.521263,0.004708,0.530099,0.024260,0.742531,0.070704,0.435965,1.147449,0.982812,0.078779,1.648568,0.000106,0.381959,1.838147,2.778348,0.963909,2.097656,2.122921,0.397906,0.254973,0.049264,2.096160,0.986901,1.494336,1.279405,0.750939,0.537910,0.255546,0.488318,1.807207,1.948368,0.087005,0.028180,0.811495,0.034125,0.658817,1.849203,0.411491,0.390414,0.413022,3.364915,1.315875,0.864022,2.546923
ARM,1.413471,0.000000,0.033031,2.271983,0.046982,0.571477,0.549905,0.734928,0.208195,0.308807,0.200285,0.119669,0.492130,0.011490,0.768603,0.386203,0.028935,3.551182,0.015564,0.153486,0.027259,0.006637,0.423467,0.012917,2.090424,0.037798,0.075696,0.006202,0.046096,0.604583,1.341131,3.047523,2.712250,3.651462,1.581337,3.674787,1.808082,0.107055,2.116432,0.279437,0.013854,0.039017,2.159639,0.009039,1.438058,0.325889,0.027852,0.228430,0.042893,0.067306,0.071894,3.311283,0.467782,0.934974,0.067038,0.038207,0.001125,0.003339,0.103895,0.207454,2.871026,3.563381,0.024158,0.042826,0.799110,1.042492,0.082981,1.008346,0.142294,0.029227,0.299668,3.289600,0.298364,0.416639,0.001745,0.067271,0.165660
AZE,1.878648,0.033031,0.000000,2.852901,0.158799,0.879289,0.852481,1.079568,0.407079,0.139847,0.395987,0.026958,0.780154,0.083484,1.120302,0.645124,0.123795,4.269188,0.093942,0.044112,0.000277,0.069280,0.219961,0.087260,2.648995,0.000161,0.208732,0.067857,0.157167,0.920243,1.795106,3.715098,3.343903,4.379072,2.071456,4.404612,2.329875,0.259016,2.678262,0.504614,0.089669,0.143845,2.726839,0.007512,1.906978,0.566422,0.000221,0.087734,0.151204,0.006036,0.007463,4.005747,0.749418,1.319475,0.005956,0.142287,0.021965,0.057375,0.254087,0.406042,3.519952,4.282562,0.000692,0.000635,1.157072,1.446651,0.220719,1.406377,0.312439,0.000116,0.531678,3.981896,0.529941,0.215048,0.049962,0.194578,0.050747
BEL,0.101388,2.271983,2.852901,0.000000,1.665538,0.564525,0.586379,0.422540,1.104655,4.256029,1.123130,3.434505,0.649298,1.960331,0.397669,0.784743,1.788125,0.142241,1.911456,3.606514,2.796968,2.033024,4.657196,1.942276,0.003780,2.895870,1.518270,2.040781,1.670841,0.532550,0.121967,0.056837,0.019483,0.162869,0.062399,0.167824,0.026458,1.392677,0.002758,0.957837,1.931004,1.715534,0.001424,2.567625,0.094938,0.876926,2.802940,3.941229,1.690530,3.121384,3.152188,0.097583,0.677928,0.291999,3.119559,1.720935,2.374206,2.101118,1.404184,1.106365,0.035006,0.144690,2.764702,2.938669,0.376236,0.236473,1.486560,0.253156,1.277105,2.816588,0.921390,0.093891,0.923681,4.634487,2.147782,1.557363,3.664634
CHN,0.945060,0.046982,0.158799,1.665538,0.000000,0.290745,0.275418,0.410274,0.057375,0.596690,0.053259,0.316614,0.234999,0.012004,0.435530,0.163782,0.002176,2.781241,0.008463,0.370303,0.145815,0.018302,0.752550,0.010629,1.510631,0.169060,0.003408,0.019044,0.000004,0.314493,0.886082,2.337728,2.045295,2.870067,1.083180,2.890750,1.272151,0.012197,1.532752,0.097260,0.009811,0.000370,1.569553,0.097234,0.965184,0.125397,0.147180,0.482602,0.000093,0.226754,0.235111,2.569417,0.218270,0.562782,0.226262,0.000453,0.062644,0.025270,0.011146,0.056986,2.183472,2.792038,0.138519,0.179519,0.458568,0.646854,0.005085,0.620017,0.025750,0.150321,0.109340,2.550322,0.108553,0.743438,0.030616,0.001816,0.389084


<a id="ID_part4"></a>
### Part 4 | Export
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [56]:
# Economic distance matrix
file_name = f"de_matrix"
s_file_export.f_df_export(df_de.reset_index(),file_name,p_file_id=f"{notebook_id}_")

Export | ../Data/1_raw_processed_backup/121_store_de_matrix_20200808_2056.csv | COMPLETE
COPY   | ../Data/2_raw_processed_input/121_input_de_matrix.csv.gzip | COMPLETE


In [57]:
# Economic distance matrix
file_name = f"di_matrix"
s_file_export.f_df_export(df_di.reset_index(),file_name,p_file_id=f"{notebook_id}_")

Export | ../Data/1_raw_processed_backup/121_store_di_matrix_20200808_2056.csv | COMPLETE
COPY   | ../Data/2_raw_processed_input/121_input_di_matrix.csv.gzip | COMPLETE


<a id="ID_part5"></a>
### Part 5
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||